## Train Probabilistic ML models

In [22]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer import SVI, Trace_ELBO
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from scipy.special import gammaln
import scipy.special
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import copy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def prepare_data(df):
    """Prepare features and target."""
    features = ['departure_latitude', 'departure_longitude', 'arrival_latitude',
                'arrival_longitude', 'departure_time_hour', 'departure_time_minute',
                'departure_time_seconds', 'departure_time_day_of_week',
                'departure_time_day_of_month', 'departure_time_month',
                'departure_time_hour_sin', 'departure_time_hour_cos',
                'departure_time_day_of_week_sin', 'departure_time_day_of_week_cos',
                'departure_time_month_sin', 'departure_time_month_cos',
                'route_distance']
    
    X = df[features].values
    X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
    y = np.maximum(df['travel_time'].values, 1e-6)
    return X, y

def evaluate_predictions(y_true, predictions):
    """Calculate only MAE, RMSE, NLL, and KL."""
    y_pred = predictions['mean']
    epsilon = 1e-8

    # Basic metrics
    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

    # NLL
    alpha = (predictions['mean'] / predictions['std'])**2
    beta = predictions['mean'] / (predictions['std']**2)
    
    alpha = np.maximum(alpha, epsilon)
    beta = np.maximum(beta, epsilon)
    y_true = np.maximum(y_true, epsilon)
    
    gamma_loglik = (alpha * np.log(beta) - 
                   scipy.special.gammaln(alpha) + 
                   (alpha - 1) * np.log(y_true) - 
                   beta * y_true)
    nll = -np.mean(gamma_loglik)

    # KL divergence
    actual_mean = np.mean(y_true)
    actual_var = np.var(y_true)
    empirical_alpha = (actual_mean ** 2) / actual_var
    empirical_beta = actual_mean / actual_var

    predicted_var = predictions['std'] ** 2
    predicted_alpha = (predictions['mean'] ** 2) / predicted_var
    predicted_beta = predictions['mean'] / predicted_var

    empirical_alpha = np.maximum(empirical_alpha, epsilon)
    empirical_beta = np.maximum(empirical_beta, epsilon)
    predicted_alpha = np.maximum(predicted_alpha, epsilon)
    predicted_beta = np.maximum(predicted_beta, epsilon)

    kl_div = (empirical_alpha * np.log(empirical_beta / predicted_beta) +
              gammaln(predicted_alpha) - gammaln(empirical_alpha) +
              (empirical_alpha - predicted_alpha) * scipy.special.digamma(empirical_alpha) +
              predicted_alpha * (empirical_beta / predicted_beta - 1))
    kl = np.mean(np.maximum(kl_div, 0.0))

    return {'mae': mae, 'rmse': rmse, 'nll': nll, 'kl': kl}

class BaseModel(BaseEstimator, RegressorMixin):
    """Base model with core metrics."""
    
    def evaluate(self, X, y):
        predictions = self.predict_distribution(X)
        metrics = evaluate_predictions(y, predictions)
        return {'predictions': predictions, 'metrics': metrics}

class LinearRegressionGamma(BaseModel):
    """Linear regression with Gamma distribution."""
    
    def __init__(self):
        self.theta = None
        self.scale = None
    
    def fit(self, X, y):
        X = X[:, -1] if X.ndim > 1 else X
        self.theta = abs(np.sum(X * y) / np.sum(X * X))
        mean_pred = self.theta * X
        residuals = y - mean_pred
        self.scale = np.var(residuals) / np.mean(mean_pred)
        return self
    
    def predict_distribution(self, X, n_samples=4000):
        X = X[:, -1] if X.ndim > 1 else X
        mean = np.maximum(self.theta * X, 1e-6)
        shape = mean / self.scale
        
        samples = np.random.gamma(shape=shape[:, np.newaxis],
                                scale=self.scale,
                                size=(len(X), n_samples))
        
        return {
            'predictions': samples,
            'mean': mean,
            'std': np.std(samples, axis=1)
        }

class RandomForestRegressorWith_Gamma(BaseModel):
    """Random Forest based distribution learning."""
    
    def __init__(self, n_estimators=100):
        self.model = RandomForestRegressor(
            n_estimators=n_estimators,
            min_samples_leaf=5,
            max_features='sqrt',
            bootstrap=True,
            random_state=42
        )
    
    def fit(self, X, y):
        self.model.fit(X, y)
        return self
    
    def predict_distribution(self, X, n_samples=4000):
        preds = np.array([est.predict(X) for est in self.model.estimators_])
        mean = np.mean(preds, axis=0)
        std = np.std(preds, axis=0)
        
        alpha = (mean / std) ** 2
        beta = mean / (std ** 2)
        samples = np.array([
            np.random.gamma(alpha[i], 1/beta[i], n_samples)
            for i in range(len(X))
        ])
        
        return {
            'predictions': samples.T,
            'mean': mean,
            'std': std
        }

class BNNTravelTimeModel(PyroModule):
    """BNN model with CUDA support."""
    
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        
        torch.set_default_dtype(torch.float32)
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        # Initialize parameters
        df = torch.tensor(9.815853992105808, dtype=torch.float32).to(device)
        loc = torch.tensor(0.0, dtype=torch.float32).to(device)
        scale = torch.tensor(1.0044249756892316, dtype=torch.float32).to(device)
      
        # First hidden layer
        self.fc1 = PyroModule[nn.Linear](input_dim, hidden_dim)
        self.fc1.weight = PyroSample(
            dist.StudentT(df=df, loc=loc, scale=scale)
            .expand([hidden_dim, input_dim]).to_event(2)
        )
        self.fc1.bias = PyroSample(
            dist.StudentT(df=df, loc=loc, scale=scale)
            .expand([hidden_dim]).to_event(1)
        )
        
        # Output layer with 2 outputs
        self.fc2 = PyroModule[nn.Linear](hidden_dim, 2)
        self.fc2.weight = PyroSample(
            dist.StudentT(df=df, loc=loc, scale=scale)
            .expand([2, hidden_dim]).to_event(2)
        )
        self.fc2.bias = PyroSample(
            dist.StudentT(df=df, loc=loc, scale=scale)
            .expand([2]).to_event(1)
        )
        
        self.relu = nn.ReLU()
        self.softplus = nn.Softplus()
        
        self = self.to(device)

    def forward(self, x, y=None):
        x = x.to(device).float()
        if y is not None:
            y = y.to(device).float()

        hidden = self.relu(self.fc1(x))
        network_output = self.fc2(hidden)
        
        log_mean = network_output[..., 0]  
        log_shape = self.softplus(network_output[..., 1])  
        
        mean = torch.exp(log_mean)
        shape = torch.exp(log_shape) + 1.0
        
        rate = torch.clamp(shape / mean, min=1e-3, max=100.0)
        shape = mean * rate
        shape = torch.clamp(shape, min=1.0)
        
        with pyro.plate("data", x.shape[0]):
            if y is not None:
                y_orig = torch.exp(y)
            else:
                y_orig = None
                
            obs = pyro.sample(
                "obs",
                dist.Gamma(shape, rate),
                obs=y_orig
            )

        return mean

class BNNPredictor(BaseEstimator, RegressorMixin):
    """BNN-based travel time predictor with CUDA support."""
    
    def __init__(self, input_dim, hidden_dim=16):
        pyro.clear_param_store()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.feature_names = [
            'departure_latitude', 'departure_longitude', 'arrival_latitude',
            'arrival_longitude', 'departure_time_hour', 'departure_time_minute',
            'departure_time_seconds', 'departure_time_day_of_week',
            'departure_time_day_of_month', 'departure_time_month',
            'departure_time_hour_sin', 'departure_time_hour_cos',
            'departure_time_day_of_week_sin', 'departure_time_day_of_week_cos',
            'departure_time_month_sin', 'departure_time_month_cos',
            'route_distance'
        ]
                    
        if input_dim != len(self.feature_names):
            raise ValueError(f"input_dim ({input_dim}) must match number of features ({len(self.feature_names)})")
            
        self.model = BNNTravelTimeModel(input_dim, hidden_dim)
        self.model.to(self.device)
        self.guide = pyro.infer.autoguide.AutoDiagonalNormal(self.model)
        self.guide.to(self.device)
        self.svi = None
        self.feature_means = None
        self.feature_stds = None

    def prepare_data(self, X, y=None):
        """Prepare data for training or prediction."""
        # Convert to DataFrame if numpy array
        if isinstance(X, np.ndarray):
            X = pd.DataFrame(X, columns=self.feature_names)
            
        features = X[self.feature_names].values
        features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
        
        # Initialize scalers during training
        if y is not None and self.feature_means is None:
            self.feature_means = np.mean(features, axis=0)
            self.feature_stds = np.std(features, axis=0)
            self.feature_stds[self.feature_stds == 0] = 1.0
        
        if self.feature_means is None:
            raise ValueError("Scalers not initialized. Need to run training first.")
        
        # Scale features
        features = (features - self.feature_means) / self.feature_stds
        features_tensor = torch.tensor(features, dtype=torch.float32).to(self.device)

        if y is not None:
            y = np.maximum(y, 1e-6)
            log_y = np.log(y)
            y_tensor = torch.tensor(log_y, dtype=torch.float32).to(self.device)
            return features_tensor, y_tensor

        return features_tensor

    def fit(self, X, y):
        """Train the model."""
        features, target = self.prepare_data(X, y)
        
        dataset = TensorDataset(features, target)
        train_loader = DataLoader(dataset, batch_size=1024, shuffle=False)

        optimizer = pyro.optim.Adam({"lr": 0.01})
        self.svi = SVI(self.model, self.guide, optimizer, loss=Trace_ELBO())
        
        best_loss = float('inf')
        best_state_dict = None
        
        try:
            for _ in range(100):
                
                perm = torch.randperm(len(features), device=self.device)
                shuffled_features = features[perm]
                shuffled_target = target[perm]
                
                shuffled_dataset = TensorDataset(shuffled_features, shuffled_target)
                loader = DataLoader(shuffled_dataset, batch_size=1024)
                
                epoch_losses = []
                for batch_X, batch_y in loader:
                    batch_X = batch_X.to(self.device)
                    batch_y = batch_y.to(self.device)
                    loss = self.svi.step(batch_X, batch_y)
                    epoch_losses.append(loss)
                
                avg_loss = np.mean(epoch_losses)
                if avg_loss < best_loss:
                    best_loss = avg_loss
                    best_state_dict = copy.deepcopy(self.model.state_dict())
            
            if best_state_dict is not None:
                self.model.load_state_dict(best_state_dict)
                
        except Exception as e:
            print(f"Training error: {str(e)}")
            raise
        
        return self
    
    def predict_distribution(self, X, n_samples=4000):
        """Generate probabilistic predictions."""
        features = self.prepare_data(X)
        
        try:
            predictive = pyro.infer.Predictive(self.model, guide=self.guide, num_samples=n_samples)
            with torch.no_grad():
                predictions = predictive(features)
            samples = predictions["obs"].cpu().detach().numpy()
            
            return {
                'predictions': samples,
                'mean': np.mean(samples, axis=0),
                'std': np.std(samples, axis=0)
            }
        except Exception as e:
            print(f"Prediction error: {str(e)}")
            raise
            
    def predict(self, X):
        """Make point predictions."""
        return self.predict_distribution(X)['mean']

    def evaluate(self, X, y):
        """Evaluate model performance."""
        predictions = self.predict_distribution(X)
        epsilon = 1e-8
        
        # Basic metrics
        mae = np.mean(np.abs(predictions['mean'] - y))
        rmse = np.sqrt(np.mean((predictions['mean'] - y) ** 2))

        # NLL calculation
        alpha = (predictions['mean'] / predictions['std'])**2
        beta = predictions['mean'] / (predictions['std']**2)
        
        alpha = np.maximum(alpha, epsilon)
        beta = np.maximum(beta, epsilon)
        y = np.maximum(y, epsilon)
        
        gamma_loglik = (alpha * np.log(beta) - 
                       scipy.special.gammaln(alpha) + 
                       (alpha - 1) * np.log(y) - 
                       beta * y)
        nll = -np.mean(gamma_loglik)

        # KL divergence
        actual_mean = np.mean(y)
        actual_var = np.var(y)
        empirical_alpha = (actual_mean ** 2) / actual_var
        empirical_beta = actual_mean / actual_var

        predicted_var = predictions['std'] ** 2
        predicted_alpha = (predictions['mean'] ** 2) / predicted_var
        predicted_beta = predictions['mean'] / predicted_var

        empirical_alpha = np.maximum(empirical_alpha, epsilon)
        empirical_beta = np.maximum(empirical_beta, epsilon)
        predicted_alpha = np.maximum(predicted_alpha, epsilon)
        predicted_beta = np.maximum(predicted_beta, epsilon)

        kl = (empirical_alpha * np.log(empirical_beta / predicted_beta) +
              scipy.special.gammaln(predicted_alpha) -
              scipy.special.gammaln(empirical_alpha) +
              (empirical_alpha - predicted_alpha) * scipy.special.digamma(empirical_alpha) +
              predicted_alpha * (empirical_beta / predicted_beta - 1))
        kl = np.mean(np.maximum(kl, 0.0))

        return {
            'mae': mae,
            'rmse': rmse,
            'nll': nll,
            'kl': kl,
            'predictions': predictions
        }
class StableGammaKDE:
    """Gamma KDE implementation."""
    
    def __init__(self, bandwidth='scott'):
        self.bandwidth_method = bandwidth
        self.bandwidth = None
        self.samples = None
    
    def fit(self, x):
        self.samples = np.maximum(x.reshape(-1), 1e-6)
        self.scale_factor = np.median(self.samples)
        self.samples = self.samples / self.scale_factor
        
        if self.bandwidth_method == 'scott':
            n = len(self.samples)
            sigma = np.std(self.samples)
            self.bandwidth = sigma * np.power(n, -1/5)
        else:
            self.bandwidth = self.bandwidth_method
        return self
    
    def sample(self, n_samples):
        idx = np.random.randint(0, len(self.samples), n_samples)
        base_samples = self.samples[idx]
        shape = np.maximum(base_samples/self.bandwidth, 1e-6)
        scale = self.bandwidth
        samples = np.random.gamma(shape, scale)
        return np.maximum(samples * self.scale_factor, 1e-6)

class ConditionalGammaKDE(BaseModel):
    """Conditional Gamma KDE model."""
    
    def __init__(self, n_clusters=50, bandwidth='scott'):
        self.n_clusters = n_clusters
        self.bandwidth = bandwidth
        self.feature_scaler = StandardScaler()
        self.clusterer = KMeans(n_clusters=n_clusters)
        self.kdes = {}
    
    def fit(self, X, y):
        X = np.asarray(X, dtype=np.float64)
        y = np.asarray(y, dtype=np.float64)
        
        X_scaled = self.feature_scaler.fit_transform(X)
        y_positive = np.maximum(y, 1e-6)
        
        self.clusterer.fit(X_scaled)
        clusters = self.clusterer.predict(X_scaled)
        
        for cluster_id in range(self.n_clusters):
            cluster_mask = (clusters == cluster_id)
            if np.sum(cluster_mask) > 10:
                cluster_times = y_positive[cluster_mask]
                kde = StableGammaKDE(bandwidth=self.bandwidth)
                kde.fit(cluster_times)
                self.kdes[cluster_id] = kde
                
        return self
    
    def predict_distribution(self, X, n_samples=4000):
        X = np.asarray(X, dtype=np.float64)
        X_scaled = self.feature_scaler.transform(X)
        clusters = self.clusterer.predict(X_scaled)
        samples = np.zeros((len(X), n_samples))
        
        for i, cluster_id in enumerate(clusters):
            if cluster_id in self.kdes:
                samples[i] = self.kdes[cluster_id].sample(n_samples)
            else:
                distances = np.linalg.norm(
                    self.clusterer.cluster_centers_ - X_scaled[i], 
                    axis=1
                )
                nearest_cluster = np.argmin(distances)
                samples[i] = self.kdes[nearest_cluster].sample(n_samples)
        
        return {
            'predictions': samples,
            'mean': np.mean(samples, axis=1),
            'std': np.std(samples, axis=1)
        }

def train_and_evaluate(model, train_df, test_df, model_name="Model"):
    """Train and evaluate model with core metrics."""
    X_train, y_train = prepare_data(train_df)
    X_test, y_test = prepare_data(test_df)
    
    model.fit(X_train, y_train)
    predictions = model.evaluate(X_test, y_test)
    metrics = predictions['metrics']
    
    print(f"\n{model_name} Performance:")
    print(f"MAE: {metrics['mae']:.2f}")
    print(f"RMSE: {metrics['rmse']:.2f}")
    print(f"NLL: {metrics['nll']:.4f}")
    print(f"KL: {metrics['kl']:.4f}")
    
    return model, predictions



def run_all_models(train_df, test_df):
    """
    Run all models and return both trained models and their results.
    
    Returns:
        tuple: (trained_models_dict, results_dict)
            - trained_models_dict: Dictionary with model names as keys and trained model objects as values
            - results_dict: Dictionary with model names as keys and evaluation results as values
    """
    initial_models = {
        'Linear Regression': LinearRegressionGamma(),
        'RandomForestRegressorWith_Gamma': RandomForestRegressorWith_Gamma(n_estimators=100),
        'BNN': BNNPredictor(input_dim=17, hidden_dim=32),
        'Conditional KDE': ConditionalGammaKDE(n_clusters=50)
    }
    
    trained_models = {}
    results = {}
    
    for name, model in initial_models.items():
        # Train and evaluate
        trained_model, preds = train_and_evaluate(model, train_df, test_df, name)
        
        # Store both the trained model and results
        trained_models[name] = trained_model
        results[name] = preds
        
    return trained_models, results

In [23]:

legs_input_path = '../Cities/Padam_terretory_01/Ressources/cleaned_padam__drt_trips_terretory_01.csv' 




crs_import = 2154 #2154 # crs of the raw imported dat
crs_working = 2154 


torch.set_default_tensor_type('torch.cuda.FloatTensor' if torch.cuda.is_available() else 'torch.FloatTensor')
pyro.set_rng_seed(0)

# Toggle for preprocessing steps
# GTFS preprocessing to get hexagonal grid
#      --> needed once, can be skipped if GTFScells.csv file exists
toggle_GTFSprep = False
GTFScells_path = '../Cities/Padam_terretory_01/Ressources/cells_of_padam_terretory_01.csv'


In [24]:
trips_df= pd.read_csv(legs_input_path, sep=",")
dataset = trips_df[['departure_latitude_x', 'departure_longitude_x', 'arrival_latitude_x',
       'arrival_longitude_x', 'departure_time_hour',
       'departure_time_minute', 'departure_time_seconds',
       'departure_time_day_of_week', 'departure_time_day_of_month',
       'departure_time_month', 'departure_time_hour_sin',
       'departure_time_hour_cos', 'departure_time_day_of_week_sin',
       'departure_time_day_of_week_cos', 'departure_time_month_sin',
       'departure_time_month_cos', 'route_distance','travel_time']]

dataset.rename(columns = {   "departure_latitude_x" : 'departure_latitude',
                            "departure_longitude_x" : "departure_longitude",
                            "arrival_latitude_x" : 'arrival_latitude',
                            "arrival_longitude_x" : 'arrival_longitude'
                        
                          },inplace=True)

dataset = dataset[dataset["travel_time"]<=2500] #consider only travel times less than 40min
dataset =dataset[dataset["travel_time"]>=60] 

/tmp/ipykernel_1836932/112659549.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.rename(columns = {   "departure_latitude_x" : 'departure_latitude',


In [30]:
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
models, results = run_all_models(train_data, test_data)

## Estimate travel times threshold between centroids

In [31]:
#read the centroids file
data = pd.read_csv("../Cities/Padam_terretory_01/Ressources/final_ready_test_data_centroids.csv")
data

,centroid_x_departure,centroid_y_departure,centroid_x_arrival,centroid_y_arrival,route_distance
0,1.826646,48.850469,1.826646,48.858257,1097.4
1,1.826646,48.850469,1.826646,48.866044,1864.0
2,1.826646,48.850469,1.826646,48.873832,3110.8
3,1.826646,48.850469,1.826646,48.881619,5822.8
4,1.826646,48.850469,1.826646,48.889406,6846.4
...,...,...,...,...,...
58801,2.050419,48.902673,2.051471,48.897194,3063.7
58802,2.050705,48.902693,2.051471,48.904981,562.4
58803,2.022691,48.914708,2.051471,48.912769,2138.0
58804,2.050953,48.925025,2.051471,48.920556,331.8


In [37]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List, Dict, Optional

def add_departure_times(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add multiple departure times to each entry in the dataframe.
    
    Args:
        df: Input DataFrame
        
    Returns:
        DataFrame with repeated entries for each departure time
    """
    departure_times = [
        '2024-12-01 07:00:00', '2024-12-01 10:00:00', '2024-12-01 12:00:00',
        '2024-12-01 17:00:00', '2024-12-01 20:00:00', '2024-12-02 07:00:00',
        '2024-12-02 10:00:00', '2024-12-02 12:00:00', '2024-12-02 17:00:00',
        '2024-12-02 20:00:00'
    ]
    
    departure_times = [pd.to_datetime(time) for time in departure_times]
    dfs = [df.assign(departure_time=time) for time in departure_times]
    return pd.concat(dfs, ignore_index=True)

def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add time-based features to the dataframe.
    
    Args:
        df: DataFrame with departure_time column
        
    Returns:
        DataFrame with additional time features
    """
    # Basic time features
    df['departure_time_hour'] = df['departure_time'].dt.hour
    df['departure_time_minute'] = df['departure_time'].dt.minute
    df['departure_time_seconds'] = df['departure_time'].dt.second
    df['departure_time_day_of_week'] = df['departure_time'].dt.dayofweek
    df['departure_time_day_of_month'] = df['departure_time'].dt.day
    df['departure_time_month'] = df['departure_time'].dt.month
    
    # Cyclical features
    df['departure_time_hour_sin'] = np.sin(2 * np.pi * df['departure_time_hour']/24)
    df['departure_time_hour_cos'] = np.cos(2 * np.pi * df['departure_time_hour']/24)
    df['departure_time_day_of_week_sin'] = np.sin(2 * np.pi * df['departure_time_day_of_week']/7)
    df['departure_time_day_of_week_cos'] = np.cos(2 * np.pi * df['departure_time_day_of_week']/7)
    df['departure_time_month_sin'] = np.sin(2 * np.pi * df['departure_time_month']/12)
    df['departure_time_month_cos'] = np.cos(2 * np.pi * df['departure_time_month']/12)
    
    return df

def prepare_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare and select features for the model.
    
    Args:
        df: Input DataFrame
        
    Returns:
        DataFrame with selected and renamed features
    """
    # Rename columns
    column_mapping = {
        'centroid_x_departure': 'departure_longitude',
        'centroid_y_departure': 'departure_latitude',
        'centroid_x_arrival': 'arrival_longitude',
        'centroid_y_arrival': 'arrival_latitude'
    }
    df = df.rename(columns=column_mapping)
    
    # Select features in correct order
    selected_features = [
        'departure_latitude', 'departure_longitude', 'arrival_latitude',
        'arrival_longitude', 'departure_time_hour', 'departure_time_minute',
        'departure_time_seconds', 'departure_time_day_of_week',
        'departure_time_day_of_month', 'departure_time_month',
        'departure_time_hour_sin', 'departure_time_hour_cos',
        'departure_time_day_of_week_sin', 'departure_time_day_of_week_cos',
        'departure_time_month_sin', 'departure_time_month_cos',
        'route_distance'
    ]
    
    return df[selected_features]

def predict_thresholds(new_data,model, confidence_levels=[0.95, 0.50]):
    """
    Predict travel time thresholds for given confidence levels.
    """

    predictions = model.predict_distribution_with_stats(new_data)
    pred_samples = predictions['predictions']

    thresholds = {}
    for conf in confidence_levels:
        upper_percentile = 100 * (1 + conf) / 2
        thresholds[conf] = {
            'threshold': np.percentile(pred_samples, upper_percentile, axis=0),
            'mean': predictions['mean'],
            'std': predictions['std']
        }

    return thresholds
def predict_travel_times(df: pd.DataFrame, predictor, features: List[str]) -> Dict:
    """
    Generate travel time predictions for each day and hour combination.
    
    Args:
        df: Input DataFrame
        predictor: Model predictor object
        features: List of feature names
        
    Returns:
        Dictionary containing predictions for each day/hour combination
    """
    days = df['departure_time_day_of_month'].unique()
    hours = df['departure_time_hour'].unique()
    daily_hourly_predictions = {}
    
    for day in days:
        print(f"Processing day {day}")
        daily_hourly_predictions[day] = {}
        
        for hour in hours:
            print(f"Predicting for day {day}, hour {hour}")
            
            day_hour_data = df[
                (df['departure_time_day_of_month'] == day) & 
                (df['departure_time_hour'] == hour)
            ]
            
            if len(day_hour_data) > 0:
                thresholds =predict_thresholds(predictor,
                    day_hour_data[features],
                    confidence_levels=[0.95, 0.70, 0.50, 0.25]
                )
                daily_hourly_predictions[day][hour] = thresholds
            else:
                print(f"No data for day {day}, hour {hour}")
                daily_hourly_predictions[day][hour] = None
                
    return daily_hourly_predictions

def assign_thresholds(df: pd.DataFrame, predictions: Dict) -> pd.DataFrame:
    """
    Assign predicted thresholds to the dataframe.
    
    Args:
        df: Input DataFrame
        predictions: Dictionary of predictions
        
    Returns:
        DataFrame with assigned thresholds
    """
    df['travel_time_95'] = None
    df['travel_time_70'] = None
    df['travel_time_50'] = None
    df['travel_time_25'] = None
    
    for day in predictions.keys():
        for hour in predictions[day].keys():
            if predictions[day][hour] is None:
                continue
                
            day_hour_mask = (df['departure_time_day_of_month'] == day) & \
                           (df['departure_time_hour'] == hour)
            day_hour_indices = df[day_hour_mask].index
            n_predictions = len(day_hour_indices)
            
            if n_predictions > 0:
                confidence_levels = [0.95, 0.70, 0.50, 0.25]
                threshold_columns = ['travel_time_95', 'travel_time_70', 
                                  'travel_time_50', 'travel_time_25']
                
                for conf, col in zip(confidence_levels, threshold_columns):
                    df.loc[day_hour_indices, col] = \
                        predictions[day][hour][conf]['threshold'][:n_predictions]
                
    return df


def process_data(data: pd.DataFrame, predictor) -> pd.DataFrame:
    """
    Process data through the complete pipeline.
    
    Args:
        data: Input DataFrame
        predictor: Model predictor object
        
    Returns:
        Processed DataFrame with predictions
    """
    # Define the features list at the module level
    FEATURES = [
        # Geographic coordinates
        'departure_latitude', 'departure_longitude',
        'arrival_latitude', 'arrival_longitude',

        # Raw time features
        'departure_time_hour',
        'departure_time_minute',
        'departure_time_seconds',
        'departure_time_day_of_week',
        'departure_time_day_of_month',
        'departure_time_month',

        # Cyclical time features
        'departure_time_hour_sin',
        'departure_time_hour_cos',
        'departure_time_day_of_week_sin',
        'departure_time_day_of_week_cos',
        'departure_time_month_sin',
        'departure_time_month_cos',

        'route_distance'
    ]
    # Add departure times and features
    data = add_departure_times(data)
    data = add_time_features(data)
    data = prepare_features(data)
    
    # Generate predictions
    predictions = predict_travel_times(data, predictor, FEATURES)
    
    # Assign predictions to dataframe
    data = assign_thresholds(data, predictions)
    
    return data

In [39]:
data=process_data(data,models["RandomForestRegressorWith_Gamma"])

##  Create GTFS files

In [ ]:
from library.create_and_join_gtfs import (
    create_gtfs_from_trips,
    save_gtfs_files,
    joinGTFS,
    exportGTFS
)

In [ ]:


# Using default threshold (travel_time_25)
gtfs_data = create_gtfs_from_trips(data,"travel_time_25")

# Or specifying a different threshold
gtfs_data_25 = create_gtfs_from_trips(data, travel_time_threshold='travel_time_25')

# Save with default threshold
save_gtfs_files(gtfs_data, "path/to/output")

# Or save with specific threshold
save_gtfs_files(gtfs_data_95, "path/to/output", travel_time_threshold='travel_time_25')

# For joining and exporting GTFS files:
merged_gtfs = joinGTFS(gtfs_dataset1, gtfs_dataset2)
exportGTFS("path/to/output", merged_gtfs)  # These functions don't need the threshold parameter